In [529]:
import math
import random # biblioteca para gerar valores aleatórios e embaralhar posição dos vetores
import copy # copiar variáveis 
import numpy as np
from scipy.stats import wrapcauchy # pegar valores da distribuição de cauchy


In [530]:
# função para criar novos possíveis caminhos
def f_embaralhar_caminho(vetor):
    vetor_embaralhado = copy.copy(vetor)  # Cria uma cópia do vetor original
    random.shuffle(vetor_embaralhado)  # Embaralha a cópia
    vetor_embaralhado.append(vetor_embaralhado[0]) # Adiciona o primeiro caminho como sendo o último
    return vetor_embaralhado

In [531]:
# função para coletar o índice dos nomes das cidades
# o resultado será utilizado para relacionar o possível caminho com a matriz de distâncias
def f_indices_vetor(vetor_alvo, vetor_referencia):
    indices = []
    for elemento in vetor_alvo:
        try:
            indice = vetor_referencia.index(elemento)
            indices.append(indice)
        except ValueError:
            indices.append(None)  
    return indices

In [532]:
# função a ser minimizada
# cálculo da distância percorrida
def f(vetor_caminho, matriz_distancias, vetor_referencia):
    vetor_index_caminho = f_indices_vetor(vetor_caminho, vetor_referencia) # colentando os índices do caminho proposto
    distancias_caminho = [] 
    for i in range(0, (len(vetor_index_caminho)-1), 1):
        valor_distancia_i = matriz_distancias[vetor_index_caminho[i]][vetor_index_caminho[i+1]] # valor da distância entre a cidade i e i+1
        distancias_caminho.append(valor_distancia_i)
        soma_total_distancia = np.sum(distancias_caminho) # somando as distâncias
    return soma_total_distancia

In [533]:
# função têmpera simulada para o problema do caixeiro viajante
def simulated_annealing(v_nome_cidades, v_solucao_inicial, m_distancia_cidades , T0, alpha, n_iterations): 
    x = v_solucao_inicial # valor inicial de x
    T = T0 # temperatura inicial, que será atualizada a cada iteração

    for k in range(n_iterations):
        # geração de outro caminho a ser percorrido
        x_prime = f_embaralhar_caminho(v_nome_cidades) 

        # cálculo das distâncias totais dos dois caminhos a serem comparados
        distancia_x1 = f(x, m_distancia_cidades, v_nome_cidades)
        distancia_x2 = f(x_prime, m_distancia_cidades, v_nome_cidades)

        # cálculo da diferença de distância
        delta_E = distancia_x2 - distancia_x1

        # aceite o novo caminho com probabilidade e^(-delta_E / T) 
        # se a diferença de energia for negativa ou a probabilidade for menor que e^(-delta_E / T), aceite o novo candidato
        if delta_E < 0 or random.random() < math.exp(-delta_E / T): 
            x = x_prime

        # Atualize a temperatura
        T = T * alpha # Atualiza a temperatura multiplicando pelo fator de resfriamento, valor entre 0 e 1, geralmente próximo de 1, com valores acima de 0.8 e menores que 0.99.

    return x

In [534]:
# exemplo ----------------------------------------------------------------------------

cidades = ["Cariacica", "Guarapari", "Serra", "Viana", "Vitoria"] # nome dos municípios

distancias = [[ 0.  ,  57. ,  29. ,  14.6 , 16. ],
              [ 57. ,  0.  ,  77.2,  42.5 , 57.8],
              [ 29. ,  77.2,  0.  ,  35.9 , 29.6],
              [ 14.6,  42.5,  35.9,  0.   , 25],
              [ 16. ,  57.8,  29.6,  25 , 0.  ]] # distância entre os municípios

solucao_inicial = ["Serra", "Guarapari", "Cariacica", "Viana", "Vitoria", "Serra"] # proposta de caminho inicial

T0 = 100.0 # Temperatura inicial
alpha = 0.8 # Fator de resfriamento
num_iterations = 10000 # Número de iterações

resultado = simulated_annealing(v_nome_cidades = cidades, v_solucao_inicial = solucao_inicial, m_distancia_cidades = distancias, T0 =T0, alpha = alpha, n_iterations = num_iterations) # resultado da simulação

# Resultados do caixeiro viajante
for indice, valor in enumerate(resultado):
    print(f"{indice + 1}º Município a ser visitado: {valor}")
print(f"Distância total percorrida em km: {round(f(resultado, distancias, solucao_inicial))}")

C:\Users\Vitoria\AppData\Local\Temp\ipykernel_20336\2320574549.py:19: RuntimeWarning: overflow encountered in scalar divide
  if delta_E < 0 or random.random() < math.exp(-delta_E / T):


1º Município a ser visitado: Vitoria
2º Município a ser visitado: Guarapari
3º Município a ser visitado: Viana
4º Município a ser visitado: Cariacica
5º Município a ser visitado: Serra
6º Município a ser visitado: Vitoria
Distância total percorrida em km: 181
